Importing the required libraries firstly.


In [2]:
import requests
from bs4 import BeautifulSoup


Then we get the html data.

In [18]:
r = requests.get("https://erasmusintern.org/traineeships", headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
c = r.content
print(c)

b'<!DOCTYPE html>\n<html lang="en" dir="ltr" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:dc="http://purl.org/dc/terms/" xmlns:foaf="http://xmlns.com/foaf/0.1/" xmlns:og="http://ogp.me/ns#" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:sioc="http://rdfs.org/sioc/ns#" xmlns:sioct="http://rdfs.org/sioc/types#" xmlns:skos="http://www.w3.org/2004/02/skos/core#" xmlns:xsd="http://www.w3.org/2001/XMLSchema#">\n<head profile="http://www.w3.org/1999/xhtml/vocab">\n<meta charset="utf-8">\n<meta name="viewport" content="width=device-width, initial-scale=1.0">\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n<script src="/cdn-cgi/apps/head/ql2_JLI0QoO4TkNybwTHy88V7W0.js"></script><link rel="shortcut icon" href="https://erasmusintern.org/profiles/erasmusintern/themes/erasmusintern_bs/favicon.ico" type="image/vnd.microsoft.icon" />\n<link rel="canonical" href="https://erasmusintern.org/traineeships" />\

All the request does is it loaded with the source code of a web page but in a very scrambled form. Now if we want to make it beautiful and extract the elements, and the text, and everything out of the source, we want to use BeautifulSoup. So all BeautifulSoup does is parsing source code and giving us what we want, giving us the elements of the HTML tags we are interested about.

In [29]:
soup = BeautifulSoup(c, "html.parser")
all_intern_divs = soup.find_all("div", {"class": "node-traineeship"} ) 

In [84]:
title, name_of_companies, field_of_studies, company_info, deadlines, duration = [], [], [], [], [], []

In [85]:
for item in all_intern_divs:
    title.append(item.find_all("h3", {"class": "dot-title"})[0].find("a").text)
    name_of_companies.append(item.find_all("div", {"class": "field-name-recruiter-name"})[0].find("a").text)
    field_of_studies.append(item.find_all("div", {"class": "ds-top-content"})[0].find("h5").text)
    company_info.append(item.find_all("div", {"class": "ds-bottom-content"})[0].find("p").text)
    deadlines.append(item.find_all("div", {"class": "field-name-field-traineeship-apply-deadline"})[0].find("span", {"class": "date-display-single"}).text)
    duration.append(item.find_all("div", {"class": "field-name-field-traineeship-duration"})[0].find("div", {"class": "field-item"}).text)

In [132]:
location = []
location_divs = soup.find_all("div", {"class": "field-name-field-traineeship-full-location"})
countries = location_divs[0]
cities = location_divs[0].find_all("div", {"class": "field-name-field-traineeship-location-city"})
for item in location_divs:
    text = ""
    countries = item.find_all("div", {"class": "field-name-field-traineeship-location-count"})
    cities = item.find_all("div", {"class": "field-name-field-traineeship-location-city"})
    for city,country in zip(cities,countries):     
        text += country.find("div", {"class":"field-item"}).text + ", " + city.find("div", {"class":"field-item"}).text + " "                 
    location.append(text)
                        
                

# Exctracting data from multiple pages

In [143]:
base_url = "https://erasmusintern.org/traineeships?page="
page_number = 6 #How many pages you want to extract data
title, name_of_companies, field_of_studies, company_info, deadlines, duration, location = [], [], [], [], [], [], []
for page in range(0, page_number):
    r = requests.get(base_url + str(page))
    c = r.content
    soup = BeautifulSoup(c, "html.parser")
    all_intern_divs = soup.find_all("div", {"class": "node-traineeship"}) 
    
    for item in all_intern_divs:
        title.append(item.find_all("h3", {"class": "dot-title"})[0].find("a").text)
        name_of_companies.append(item.find_all("div", {"class": "field-name-recruiter-name"})[0].find("a").text)
        field_of_studies.append(item.find_all("div", {"class": "ds-top-content"})[0].find("h5").text)
        company_info.append(item.find_all("div", {"class": "ds-bottom-content"})[0].find("p").text)
        deadlines.append(item.find_all("div", {"class": "field-name-field-traineeship-apply-deadline"})[0].find("span", {"class": "date-display-single"}).text)
        duration.append(item.find_all("div", {"class": "field-name-field-traineeship-duration"})[0].find("div", {"class": "field-item"}).text)
    
    location_divs = soup.find_all("div", {"class": "field-name-field-traineeship-full-location"})
    countries = location_divs[0]
    cities = location_divs[0].find_all("div", {"class": "field-name-field-traineeship-location-city"})
    for item in location_divs:
        text = ""
        countries = item.find_all("div", {"class": "field-name-field-traineeship-location-count"})
        cities = item.find_all("div", {"class": "field-name-field-traineeship-location-city"})
        for city,country in zip(cities,countries):     
            text += country.find("div", {"class":"field-item"}).text + ", " + city.find("div", {"class":"field-item"}).text + " "                 
        location.append(text)

## Saving the extracted data in CSV

In [146]:
import pandas as pd
l = {"Title":title,
     "Company":name_of_companies,
     "Company's info":company_info, 
     "Field of Studies":field_of_studies,
     "Duration of Internship":duration,
     "Deadline" : deadlines,
     "Location(s)" : location
    }
df = pd.DataFrame(l)

In [147]:
df

,Title,Company,Company's info,Field of Studies,Duration of Internship,Deadline,Location(s)
0,COPYWRITER REMOTE OR ON SITE,Conoce Galicia SL,This is not a job offer. This is an internship...,Languages and Philological Sciences,3 months,"31 Mar, 2021","Spain, Santiago de Compostela"
1,Administration and accounting intern,Internship Centre,Our collaborator is Law company in Sevilla. Cu...,"Communication and Information Sciences, Langua...",9 months,"31 Mar, 2021","Spain, BArcelona, Madrid , Sevilla or online"
2,Legal and administration intern,Internship Centre,Our collaborator is Law company in Sevilla. Cu...,"Communication and Information Sciences, Langua...",9 months,"31 Mar, 2021","Spain, BArcelona, Madrid , Sevilla or online"
3,Business and adminsitration intern,Internship Centre,Our collaborator is Law company in Sevilla. Cu...,"Communication and Information Sciences, Langua...",9 months,"31 Mar, 2021","Spain, BArcelona, Madrid , Sevilla or online"
4,"Human Resources intern in Barcelona, Madrid or...",Internship Centre,Currently for our client we are looking for ne...,"Art and Design, Humanities, Languages and Phil...",8 months,"31 Mar, 2021","Spain, BArcelona, Madrid or online"
5,"Recruiter intern in Barcelona, Madrid or online",Internship Centre,Currently for our client we are looking for ne...,"Art and Design, Humanities, Languages and Phil...",8 months,"31 Mar, 2021","Spain, BArcelona, Madrid or online"
6,Finance intern in Barcelona or online,Internship Centre,Internship at financial analysis department (m...,"Art and Design, Business Studies and/or Manage...",9 months,"31 Mar, 2021","Spain, Barcelona or online"
7,Business development intern in Barcelona or on...,Internship Centre,Our collaborator is a leading creative agency ...,"Art and Design, Business Studies and/or Manage...",10 months,"31 Mar, 2021","Spain, Barcelona or online"
8,Advert spec. intern in Barcelona or online,Internship Centre,Our collaborator is a leading creative agency ...,"Art and Design, Business Studies and/or Manage...",10 months,"31 Mar, 2021","Spain, Barcelona or online"
9,Marketing intern in Barcelona or online,Internship Centre,Our collaborator is a leading creative agency ...,"Art and Design, Business Studies and/or Manage...",10 months,"31 Mar, 2021","Spain, Barcelona or online"


In [148]:
df.to_csv("Output.csv")